# Pre-Processing

In [5]:
from google.colab import drive
from google.colab import files

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
import re
import csv
import sys
import uuid
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # Bag of Words: Term Frequencey and TFIDF
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

# Regex Expressions
REGEX_PATTERNS = {
    'currency': r'[$€£¥]\s*\d+(?:[.,]\d+)?|\d+(?:[.,]\d+)?\s*(?:USD|EUR|GBP|JPY|CAD|AUD|CHF)',

    'time': r'\b(?:[01]?\d|2[0-3]):[0-5]\d(?::[0-5]\d)?(?:\s*[aApP][mM])?\b',

    'day' : r'(?i) (sun|mon|tue(s)?|wed(nesday)?|thu(r(s)?)?|fri)(day|\.)? ', # note the spaces at the beginning and end

    'date': r'(?i)\b(?:\d{1,2}[-\/\.]\d{1,2}[-\/\.]\d{2,4}|\d{4}[-\/\.]\d{1,2}[-\/\.]\d{1,2}|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s+\d{1,2}(?:[a-z]{2})?,?\s+\d{2,4}|\d{1,2}\s+(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*,?\s+\d{2,4}|(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*\s+\d{1,2}(?:\s?[—-]\s?\d{1,2})?)\b',

    'phone': r'^(\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]\d{3}[\s.-]\d{4}$',

    'percentage': r'\b(?<!\.)(?!0+(?:\.0+)?%)(?:\d|[1-9]\d|100)(?:(?<!100)\.\d+)?%',

    'number': r'\b\d+(?:[.,]\d+)?\b',

    'email': r'(?:[a-z0-9!#$%&\'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&\'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])',

    'url' : r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
}

## Load Dataset(s)

In [7]:

pd.set_option('display.max_colwidth', None)
csv.field_size_limit(sys.maxsize)

file_paths = {'/content/drive/MyDrive/COS720 Project/Datasets/CEAS_08.csv',
              '/content/drive/MyDrive/COS720 Project/Datasets/Nigerian_Fraud.csv',
              '/content/drive/MyDrive/COS720 Project/Datasets/Ling.csv'}

def load_email_data(file_path):

  with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
        reader = csv.reader(f, quotechar='"', escapechar='\\')
        headers = next(reader)
        data = []
        for row in reader:
            if len(row) == len(headers):
                data.append(row)
            else:
                print(f"Skipping malformed row: {row}")

        df = pd.DataFrame(data, columns=headers)
        print(f"Loaded data with fallback method: {len(df)} rows")
        return df

## Clean Data

### Extract emails from body

### Converting to lowercase

In [8]:
def to_lowercase(df, text_column='body'):
  df_copy = df.copy()

  df_copy[text_column] = df_copy[text_column].apply(
      lambda x: x.lower() if isinstance(x, str) else x
  )

  return df_copy

### Replacing text with classes

In [9]:
## we replace currency with <CUR>, time with <TIME>, date with <DATE>, phone numbers with <PHONE>, percentages with <PERC>, and other general numbers that do not fall into the previous categories with <NUM>

def replace_data_categories(text):
    modified_text = text

    # Replace each type in order (specific to general)
    modified_text = re.sub(REGEX_PATTERNS['currency'], '<cur>', modified_text) # currency
    modified_text = re.sub(REGEX_PATTERNS['time'], '<time>', modified_text) # times
    modified_text = re.sub(REGEX_PATTERNS['day'], '<day>', modified_text) # times
    modified_text = re.sub(REGEX_PATTERNS['date'], '<date>', modified_text) # dates
    modified_text = re.sub(REGEX_PATTERNS['phone'], '<phone>', modified_text) # phone numbers
    modified_text = re.sub(REGEX_PATTERNS['percentage'], '<perc>', modified_text) # percentages
    modified_text = re.sub(REGEX_PATTERNS['number'], '<num>', modified_text) # general numbers

    return modified_text

### Extract emails from body

In [10]:
def extract_urls_from_text(text):
    if not isinstance(text, str):
        return []

    # Regex pattern to match URLs
    url_pattern = re.compile(REGEX_PATTERNS['url'])
    return url_pattern.findall(text)

def replace_urls_with_tag(text):
    if not isinstance(text, str):
        return text

    url_pattern = re.compile(REGEX_PATTERNS['url'])
    return url_pattern.sub('<URL>', text) #  Replace all URLs in text with <URL> tag


def process_email_data(df):

    print(f"Original dataframe shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")

    url_data = []

    # ID to link URLs back to emails
    df['email_id'] = [str(uuid.uuid4()) for _ in range(len(df))]

    # Extract and replace URLs -- Subject to further processing
    for idx, row in df.iterrows():
        email_id = row['email_id']
        body = row['body']

        urls = extract_urls_from_text(body)

        for url in urls:
            url_data.append({
                'email_id': email_id,
                'url': url,
                'sender': row['sender'],
                'label': row['label']
            })

        # Replace urls
        df.at[idx, 'body'] = replace_urls_with_tag(body)

        # Replace data categories
        df.at[idx, 'body'] = replace_data_categories(row['body'])

    # Lowercase
    df = to_lowercase(df)

    # Create URL dataframe
    url_df = pd.DataFrame(url_data)

    print(f"Modified dataframe shape: {df.shape}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"URL dataframe shape: {url_df.shape}")

    return df, url_df

data_frames = []

for file_path in file_paths:
     data_frames.append(load_email_data(file_path))

combined_df = pd.concat(data_frames, join='outer', ignore_index=True)
combined_df, url_df = process_email_data(combined_df)

combined_df['label'] = combined_df['label'].astype(int)
print(combined_df.shape)
combined_df

Loaded data with fallback method: 2859 rows
Loaded data with fallback method: 39154 rows
Loaded data with fallback method: 3332 rows
Original dataframe shape: (45345, 7)
Columns: ['subject', 'body', 'label', 'sender', 'receiver', 'date', 'urls']
Modified dataframe shape: (45345, 8)
Columns: ['subject', 'body', 'label', 'sender', 'receiver', 'date', 'urls', 'email_id']
URL dataframe shape: (132701, 4)
(45345, 8)


subject  \
0                 job posting - apple-iss research center   
1                                                           
2      query : letter frequencies for text identification   
3                                                    risk   
4                                request book information   
...                                                   ...   
45340                  CONTACT GLOBAL MAX SHIPING COMPANY   
45341                                    TREAT AS URGENT.   
45342    From Dr Usman Ibrahim / Mr Wahid Yoffe property.   
45343                             My Beloved In Christ.     
45344                                           ATTENTION   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

### Splitting data

In [19]:
# Split data into test and train subsets
X_train, X_test, target_train, target_test = train_test_split(combined_df['body'], combined_df['label'], train_size=0.7, random_state=1)
print(target_test.head(10))

45258    1
22057    1
21931    0
43867    1
36030    1
10926    1
29301    1
22653    1
44468    1
44120    1
Name: label, dtype: int64


### Removing non-word and non special characters

### Tokenization and Vectorization

In [20]:
# Initialize vecotrizer
def initialise_tfidf_vectorizer(data, max_features = 2000):
    vectorizer_tfidf = TfidfVectorizer(stop_words='english', max_features=max_features) # remove stop words
    vectorizer_tfidf.fit(data)
    X = vectorizer_tfidf.transform(data)
    return X, vectorizer_tfidf

X_train_tfidf, vectorizer_tfidf = initialise_tfidf_vectorizer(X_train)

### Lemmatization

### Train/Import and Test Model(s)

In [21]:
from sklearn.neural_network import MLPClassifier
import joblib

MODEL_PATH = "/content/drive/MyDrive/COS720 Project/Models/body_classifier.joblib"

try:
    # Try loading existing model
    clf = joblib.load(MODEL_PATH)
    print("Loaded existing model from", MODEL_PATH)
except FileNotFoundError:
    print("Model file not found. Training a new model...")
    clf = MLPClassifier(hidden_layer_sizes=(100,),  # one hidden layer with 100 units
                        activation='logistic',
                        solver='adam',
                        max_iter=200,
                        random_state=0)

    # Train the model
    clf.fit(X_train_tfidf, target_train)

# scores = cross_val_score(
#     clf,
#     X_train_tfidf,
#     target_train,
#     cv=5,
#     scoring='f1'
# )

# print("5-fold Cross-Validation F1 score for Word Frequency: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Loaded existing model from /content/drive/MyDrive/COS720 Project/Models/body_classifier.joblib


In [22]:
# from sklearn.pipeline import make_pipeline
# import joblib

# # Build the pipeline (no SVD)
# pipeline = make_pipeline(
#     vectorizer_tfidf,  # fitted TfidfVectorizer
#     clf                # trained classifier
# )

# # Save the pipeline
# joblib.dump(pipeline, 'simple_text_classifier.joblib')

### Testing

In [34]:
test_file_path = "/content/drive/MyDrive/COS720 Project/Datasets/SpamAssasin.csv"

# Evaluate model using different data file from training
with open(test_file_path, 'r', encoding='utf-8', errors='replace') as f:
  reader = csv.reader(f, quotechar='"', escapechar='\\')
  headers = next(reader)
  data = []
  for row in reader:
      if len(row) == len(headers):
          data.append(row)
  test_df = pd.DataFrame(data, columns=headers)
  test_df['label'] = test_df['label'].astype(int)
  print(f"Loaded data with fallback method: {len(test_df)} rows")

test_df, test_url_df = process_email_data(test_df)

# y_pred = clf.predict(X_test)
# y_test = target_test

y_test = test_df['label']
y_pred = clf.predict(test_df['body'])

f1 = f1_score(y_test, y_pred)  # by default, pos_label=1
print(f"F1 Score: {f1:.4f}")

Loaded data with fallback method: 5805 rows
Original dataframe shape: (5805, 7)
Columns: ['sender', 'receiver', 'date', 'subject', 'body', 'label', 'urls']
Modified dataframe shape: (5805, 8)
Columns: ['sender', 'receiver', 'date', 'subject', 'body', 'label', 'urls', 'email_id']
URL dataframe shape: (23462, 4)
F1 Score: 0.8091
